In [1]:
#import libraries
import os
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree, metrics
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


In [42]:
sports_data_21=pd.read_csv('/content/drive/MyDrive/MIDSEM PROJECT/players_21.csv')#train

In [43]:
sports_data_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Columns: 110 entries, sofifa_id to nation_flag_url
dtypes: float64(16), int64(44), object(50)
memory usage: 15.9+ MB


Drop features with 30% NaN

In [44]:
threshold = 0.30 * len(sports_data_21)

# Identify columns with 30% or more NaN values
columns_with_nan = sports_data_21.columns[sports_data_21.isna().sum() >= threshold]

# Drop the identified columns
sports_data_21.drop(columns=columns_with_nan, inplace=True)

Getting numerical and object columns

In [45]:
numerical_columns = sports_data_21.select_dtypes(include=['int64', 'float64'])

In [46]:
object_columns = sports_data_21.select_dtypes(include=['object']).columns

In [47]:
object_columns

Index(['player_url', 'short_name', 'long_name', 'player_positions', 'dob',
       'club_name', 'league_name', 'club_position', 'club_joined',
       'nationality_name', 'preferred_foot', 'work_rate', 'body_type',
       'real_face', 'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam',
       'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm',
       'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk', 'player_face_url',
       'club_logo_url', 'club_flag_url', 'nation_flag_url'],
      dtype='object')

Encoding object columns

In [48]:
for column in object_columns:
    sports_data_21[column] = pd.factorize(sports_data_21[column])[0]


Imputing  columns

In [49]:
# Create a SimpleImputer instance with your chosen strategy
imputer = SimpleImputer(strategy='mean')

# Iterate over columns and impute
for column in sports_data_21:
    sports_data_21[column] = imputer.fit_transform(sports_data_21[[column]])


In [50]:
sports_data_21.isna().any() #Check if there are any NaNs

sofifa_id           False
player_url          False
short_name          False
long_name           False
player_positions    False
                    ...  
gk                  False
player_face_url     False
club_logo_url       False
club_flag_url       False
nation_flag_url     False
Length: 102, dtype: bool

In [51]:
c = sports_data_21.corr()
c["overall"].sort_values(ascending=False)

overall                1.000000
movement_reactions     0.867234
mentality_composure    0.705252
passing                0.662090
potential              0.636366
                         ...   
dob                   -0.595900
short_name            -0.923514
long_name             -0.974388
player_face_url       -0.975045
player_url            -0.975045
Name: overall, Length: 102, dtype: float64

In [52]:
#Using correlation values greater than 0.4 we selected these
s =['overall','movement_reactions','passing','mentality_composure','dribbling','release_clause_eur','wage_eur','value_eur','power_shot_power','physic','mentality_vision',
           'attacking_short_passing','shooting','skill_long_passing','age','skill_ball_control','international_reputation','skill_curve','attacking_crossing']

In [53]:
#We picked the top 10
selected =['overall','movement_reactions','passing','dribbling','age','mentality_composure','release_clause_eur','power_shot_power','wage_eur','value_eur']

In [54]:
sports_data_21 = sports_data_21[selected]

In [55]:
c = sports_data_21.corr()
c["overall"].sort_values(ascending=False)

overall                1.000000
movement_reactions     0.867234
mentality_composure    0.705252
passing                0.662090
release_clause_eur     0.614344
dribbling              0.596558
wage_eur               0.589060
power_shot_power       0.558372
value_eur              0.555192
age                    0.468197
Name: overall, dtype: float64

In [56]:
sports_data_21.head()

,overall,movement_reactions,passing,dribbling,age,mentality_composure,release_clause_eur,power_shot_power,wage_eur,value_eur
0,93.0,94.0,91.0,95.0,33.0,96.0,138400000.0,86.0,560000.0,103500000.0
1,92.0,95.0,81.0,89.0,35.0,95.0,75900000.0,94.0,220000.0,63000000.0
2,91.0,93.0,78.0,85.0,31.0,88.0,132000000.0,89.0,240000.0,111000000.0
3,91.0,91.0,86.0,94.0,28.0,93.0,166500000.0,80.0,270000.0,132000000.0
4,91.0,91.0,93.0,88.0,29.0,91.0,161000000.0,91.0,370000.0,129000000.0


Scaling

In [57]:
from sklearn.preprocessing import StandardScaler

In [58]:
y = sports_data_21['overall']


In [59]:
x =sports_data_21.drop('overall',axis=1)

In [60]:
X= StandardScaler()

In [61]:
scaled = X.fit_transform(x)

In [62]:
sports_data_21=pd.DataFrame(scaled,columns=x.columns)

In [63]:
sports_data_21

,movement_reactions,passing,dribbling,age,mentality_composure,release_clause_eur,power_shot_power,wage_eur,value_eur
0,3.554438,3.493625,3.433013,1.655055,3.137573,13.695696,2.119026,27.857178,13.073165
1,3.664174,2.461857,2.800094,2.080838,3.055051,7.312715,2.719163,10.662997,7.809992
2,3.444701,2.152326,2.378148,1.229273,2.477402,13.042079,2.344077,11.674420,14.047827
3,3.225227,2.977741,3.327526,0.590598,2.890009,16.565484,1.668922,13.191553,16.776879
4,3.225227,3.699979,2.694607,0.803490,2.724966,16.003782,2.494112,18.248665,16.387015
...,...,...,...,...,...,...,...,...,...
18939,-1.493455,-3.212867,-3.740069,-0.899642,-1.483625,-0.432955,-1.706851,-0.412078,-0.368070
18940,-1.273981,-0.839801,-1.630339,-0.899642,-1.896232,-0.431423,-1.481799,-0.412078,-0.368070
18941,-1.932402,-0.839801,-1.735826,0.590598,-1.896232,-0.433977,-0.656610,-0.361507,-0.371319
18942,-0.944771,-1.768392,-0.997420,-1.751207,-1.071018,-0.421926,-0.581593,-0.437363,-0.360272


In [64]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


Training with Random Forest Regressor

In [238]:
rfm = RandomForestRegressor(n_estimators=150, random_state=45)
rfm.fit(X_train, y_train)


RandomForestRegressor(n_estimators=150, random_state=45)

In [239]:
scores = cross_val_score(rfm, X_train, y_train, cv=10)
print("Cross-Validation Scores:", scores)


Cross-Validation Scores: [0.99132243 0.99003622 0.98712724 0.9891653  0.99044148 0.99091784
 0.98894771 0.98899937 0.98700807 0.9875041 ]


In [240]:
y_pred = rfm.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 0.48032677047593914


In [165]:
mae = print(mean_absolute_error(y_test, y_pred))

0.4209342834520982


Training with Decision Tree Classifier

In [112]:
from sklearn.tree import DecisionTreeRegressor

In [178]:
dtree = DecisionTreeRegressor(max_depth=20, min_samples_split=6)

In [179]:
dtree.fit(X_train,y_train)

DecisionTreeRegressor(max_depth=20, min_samples_split=6)

In [180]:
y_pred = dtree.predict(X_test)

In [116]:
scores = cross_val_score(dtree, X_train, y_train, cv=10)
print("Cross-Validation Scores:", scores)


Cross-Validation Scores: [0.98304544 0.98299906 0.97306702 0.98203085 0.98413469 0.98254015
 0.98040883 0.98226193 0.97831501 0.9774824 ]


In [181]:
mae = print(mean_absolute_error(y_test, y_pred))

0.47795590685307743


In [118]:
from sklearn.metrics import accuracy_score,f1_score,make_scorer

Training using Gradient Boost


In [182]:
from sklearn.ensemble import GradientBoostingRegressor

In [197]:
grad=GradientBoostingRegressor( n_estimators=170, learning_rate=0.5)

In [198]:
grad.fit(X_train,y_train)

GradientBoostingRegressor(learning_rate=0.5, n_estimators=170)

In [199]:
y_pred=grad.predict(X_test)

In [200]:
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 0.5558144673996569


In [150]:
scores = cross_val_score(grad, X_train, y_train, cv=10)
print("Cross-Validation Scores:", scores)


Cross-Validation Scores: [0.98884129 0.98731696 0.98580095 0.98474563 0.98792089 0.98847779
 0.98608635 0.98787075 0.98462785 0.98396308]


In [201]:
mae = print(mean_absolute_error(y_test, y_pred))

0.5344814153767615


Voting Regressor

In [202]:
from sklearn.ensemble import VotingRegressor

In [203]:
# Create a VotingRegressor instance
voting_regressor = VotingRegressor(
    estimators=[
        ('gradboost', grad),
        ('rf', rfm),
        ('decision tree', dtree)
    ],
n_jobs=-1,
weights=[1, 1, 1]
)


In [204]:
for model in (grad, rfm,dtree,voting_regressor):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Calculate Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_test, y_pred)

    # Print the model's name and MAE
    print(model.__class__.__name__, "MAE:", mae)

GradientBoostingRegressor MAE: 0.5346882406124349
RandomForestRegressor MAE: 0.4209342834520982
DecisionTreeRegressor MAE: 0.4793810850003458
VotingRegressor MAE: 0.4389934526892038


Optimization Of Models

In [205]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold


In [216]:
cv = KFold(n_splits=6)
# Define the parameter grid
PARAMETERS = {
    "max_depth": [2, 5, 6, 12],
    "min_samples_split": [2, 5, 7,10],
    "n_estimators": [100,120, 150]
}




In [217]:
def custom_scorer(y_true, y_pred):
    return -mean_absolute_error(y_true, y_pred)  # Using negative MAE for GridSearchCV


In [218]:
rfm = RandomForestRegressor(n_jobs=-1)

scoring = make_scorer(mean_squared_error, greater_is_better=False)


In [219]:
# Create a GridSearchCV instance
model_gs = GridSearchCV(
    rfm,
    param_grid=PARAMETERS,
    cv=cv,
    scoring=scoring,
    verbose=0
)


In [220]:
# Fit the model to the training data
model_gs.fit(X_train, y_train)

# Make predictions
y_pred = model_gs.predict(X_test)

Testing with Players 22

In [243]:
sports_data_22=pd.read_csv('/content/drive/MyDrive/MIDSEM PROJECT/players_22.csv')

<ipython-input-243-758a12422fcc>:1: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  sports_data_22=pd.read_csv('/content/drive/MyDrive/MIDSEM PROJECT/players_22.csv')


In [244]:
threshold = 0.30 * len(sports_data_22)

# Identify columns with 30% or more NaN values
columns_with_nan = sports_data_22.columns[sports_data_22.isna().sum() >= threshold]

# Drop the identified columns
sports_data_22.drop(columns=columns_with_nan, inplace=True)

In [245]:
num_columns = sports_data_22.select_dtypes(include=['int64', 'float64'])

In [246]:
obj_columns = sports_data_22.select_dtypes(include=['object']).columns

In [247]:
for column in obj_columns:
    sports_data_22[column] = pd.factorize(sports_data_22[column])[0]


In [248]:

# Create a SimpleImputer instance with your chosen strategy
imputer = SimpleImputer(strategy='mean')

# Iterate over columns and impute
for column in sports_data_22:
    sports_data_22[column] = imputer.fit_transform(sports_data_22[[column]])


In [249]:
#We picked the top 10
selected_data =['overall','movement_reactions','passing','dribbling','age','mentality_composure','release_clause_eur','power_shot_power','wage_eur','value_eur']

In [250]:
sports_data_22 = sports_data_22[selected_data]

In [251]:
y_new= sports_data_22['overall']


In [252]:
x_new = sports_data_22.drop('overall',axis=1)

In [253]:

sc= StandardScaler()

In [254]:
scaled_new = sc.fit_transform(x_new)

In [255]:
sports_data_22=pd.DataFrame(scaled_new,columns=x_new.columns)

In [256]:
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(x_new, y_new, test_size=0.2, random_state=42)


Testing with random forest regressor

In [261]:
# Now you can make predictions on the imputed data
y_pred_new = rfm.predict(x_new)


In [262]:
maeee = print(mean_absolute_error(y_new, y_pred_new))

0.7076712233830587


Testing with Decision Tree Regressor

In [263]:
# Now you can make predictions on the imputed data
y_pred_new = dtree.predict(x_new)


In [264]:
maee = print(mean_absolute_error(y_new, y_pred_new))

0.7980615901459159


Testing with Gradient Boost Regressor

In [265]:
# Now you can make predictions on the imputed data
y_pred_new = grad.predict(x_new)


In [266]:
maeee = print(mean_absolute_error(y_new, y_pred_new))

0.6923099046281611


Saving the Random Forest Regressor

In [267]:
import pickle
filename = 'finalized_best_model.pkl'
pickle.dump(rfm, open(filename, 'wb'))
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
y_pred = loaded_model.predict(X_test)
print(y_pred)

[59.98666667 63.92       58.54       ... 72.29333333 62.74
 61.22      ]
